# Семантический Агент — Аналитика через Cube

Задавайте вопросы на естественном языке и получайте данные из Cube.

**Перед запуском убедитесь что:**
1. Заполнен `config.yml`
2. Запущен Cube (`npx cubejs-server`)
3. Выполнен `python 01_data_loader.py` (модели загружены в Cube)
4. Выполнен `python 02_build_faiss.py` (FAISS-индекс построен)

> Первая ячейка автоматически проверит и установит все зависимости (FAISS, sentence-transformers, и т.д.)

In [ ]:
# ============================================================
# АВТОУСТАНОВКА ЗАВИСИМОСТЕЙ (выполнить первой!)
# ============================================================
# Если пакеты уже установлены — ячейка пройдёт мгновенно.
# Если нет — автоматически скачает и установит всё необходимое.

import sys, subprocess

def _ensure_packages():
    """Проверить и установить недостающие пакеты"""
    required = {
        "yaml": "pyyaml",
        "httpx": "httpx",
        "faiss": "faiss-cpu",
        "langchain_core": "langchain-core",
        "langchain_community": "langchain-community",
        "langchain": "langchain",
        "langchain_gigachat": "langchain-gigachat",
        "sentence_transformers": "sentence-transformers",
        "tabulate": "tabulate",
        "pandas": "pandas",
    }
    missing = []
    for module, pip_name in required.items():
        try:
            __import__(module)
        except ImportError:
            missing.append(pip_name)

    # PyTorch — проверяем отдельно, ставим CPU-версию
    try:
        import torch
    except ImportError:
        missing.append("torch")

    if not missing:
        print("✅ Все зависимости уже установлены")
        return

    print(f"📦 Недостающие пакеты: {', '.join(missing)}")
    cmd = [sys.executable, "-m", "pip", "install", "--quiet"]

    # Torch CPU отдельно (экономим ~2 ГБ vs CUDA-версия)
    if "torch" in missing:
        print("   ⚡ Установка PyTorch (CPU)...")
        subprocess.check_call(
            cmd + ["torch", "--index-url", "https://download.pytorch.org/whl/cpu"]
        )
        missing.remove("torch")

    if missing:
        print(f"   📥 Установка: {', '.join(missing)}...")
        subprocess.check_call(cmd + missing)

    print("✅ Все пакеты установлены!")

_ensure_packages()

In [ ]:
# ============================================================
# ИНИЦИАЛИЗАЦИЯ (выполнить один раз)
# ============================================================

import os, json, re, yaml, time, httpx
from pathlib import Path
from dataclasses import dataclass, field
from typing import List, Dict, Optional, Any

# Загрузка конфигурации
with open("config.yml", 'r', encoding='utf-8') as f:
    CONFIG = yaml.safe_load(f)

CUBE_URL = CONFIG["cube"]["api_url"]
CUBE_TOKEN = CONFIG["cube"].get("api_token", "")
FAISS_PATH = CONFIG["faiss"]["index_path"]
EMBEDDING_MODEL = CONFIG["faiss"]["embedding_model"]
SEARCH_K = CONFIG["faiss"].get("search_k", 20)
MAX_ROWS = CONFIG["agent"].get("max_rows_display", 15)

print("✅ Конфигурация загружена")
print(f"   Cube URL: {CUBE_URL}")
print(f"   FAISS path: {FAISS_PATH}")

In [ ]:
# ============================================================
# ЗАГРУЗКА КОМПОНЕНТОВ
# ============================================================

from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_gigachat import GigaChat

# 1. Эмбеддинги и FAISS
print("🔄 Загрузка модели эмбеддингов...")
embeddings = HuggingFaceEmbeddings(model_name=EMBEDDING_MODEL)
print("🔄 Загрузка FAISS-индекса...")
vector_store = FAISS.load_local(FAISS_PATH, embeddings, allow_dangerous_deserialization=True)

# 2. Метаданные members
with open(os.path.join(FAISS_PATH, "members.json"), 'r', encoding='utf-8') as f:
    ALL_MEMBERS = json.load(f)

# Карта name → title для русских названий
TITLE_MAP = {}
for m in ALL_MEMBERS:
    TITLE_MAP[m["name"]] = m["title"]
    short = m["name"].split(".")[-1]
    if short not in TITLE_MAP:
        TITLE_MAP[short] = m["title"]

# 3. GigaChat — поддержка двух режимов подключения
gc = CONFIG["gigachat"]
if gc.get("base_url"):
    # Режим 2: через прокси (закрытый контур)
    token_env = gc.get("access_token_env", "JPY_API_TOKEN")
    llm = GigaChat(
        base_url=gc["base_url"],
        access_token=os.getenv(token_env, ""),
        model=gc.get("model", "GigaChat")
    )
elif gc.get("credentials"):
    # Режим 1: через credentials (SberCloud)
    llm = GigaChat(
        credentials=gc["credentials"],
        model=gc.get("model", "GigaChat"),
        verify_ssl_certs=gc.get("verify_ssl", False),
        timeout=gc.get("timeout", 60)
    )
else:
    raise RuntimeError("Заполните gigachat.credentials или gigachat.base_url в config.yml")

# 4. Semantic config (glossary + examples) — пути из конфига
sem = CONFIG.get("semantic", {})
glossary_path = Path(sem.get("glossary_path", "config/glossary.yml"))
examples_path = Path(sem.get("examples_path", "config/examples.yml"))

GLOSSARY = {}
if glossary_path.exists():
    with open(glossary_path, 'r', encoding='utf-8') as f:
        GLOSSARY = yaml.safe_load(f) or {}

EXAMPLES = []
if examples_path.exists():
    with open(examples_path, 'r', encoding='utf-8') as f:
        EXAMPLES = yaml.safe_load(f) or []

print(f"✅ Всё загружено!")
print(f"   FAISS members: {len(ALL_MEMBERS)}")
print(f"   Glossary terms: {len(GLOSSARY)}")
print(f"   Examples: {len(EXAMPLES)}")
print(f"   LLM: GigaChat ({gc.get('model', 'GigaChat')})")

In [ ]:
# ============================================================
# ЯДРО АГЕНТА
# ============================================================

def search_relevant(question: str, k: int = SEARCH_K) -> List[Dict]:
    """Семантический поиск по FAISS"""
    results = vector_store.similarity_search_with_score(question, k=k)
    return [
        {
            "name": doc.metadata["name"],
            "title": doc.metadata["title"],
            "type": doc.metadata["type"],
            "cube_name": doc.metadata["cube_name"],
            "member_type": doc.metadata["member_type"],
            "agg_type": doc.metadata.get("agg_type", ""),
            "description": doc.metadata.get("description", ""),
            "score": float(score)
        }
        for doc, score in results
    ]


def find_relevant_examples(question: str, limit: int = 5) -> List[Dict]:
    """Найти подходящие few-shot примеры"""
    q_lower = question.lower()
    q_words = set(q_lower.split())
    scored = []
    for ex in EXAMPLES:
        ex_words = set(ex.get("question", "").lower().split())
        overlap = len(q_words & ex_words)
        tag_score = sum(2 for t in ex.get("tags", []) if t in q_lower)
        score = overlap * 0.5 + tag_score
        if score > 0:
            scored.append((score, ex))
    scored.sort(key=lambda x: x[0], reverse=True)
    return [ex for _, ex in scored[:limit]]


def build_prompt(question: str, relevant: List[Dict]) -> str:
    """Построить промпт для LLM"""
    # Меры
    measures = []
    for m in relevant:
        if m["member_type"] == "measure":
            line = f"- {m['name']}: {m['title']}"
            if m["agg_type"]:
                line += f" ({m['agg_type']})"
            if m["description"]:
                line += f" — {m['description'][:100]}"
            measures.append(line)
    
    # Измерения
    dimensions = []
    for m in relevant:
        if m["member_type"] == "dimension":
            line = f"- {m['name']}: {m['title']}"
            if m["description"]:
                line += f" — {m['description'][:100]}"
            dimensions.append(line)
    
    # Примеры
    examples = find_relevant_examples(question)
    examples_text = ""
    for i, ex in enumerate(examples, 1):
        q_str = json.dumps(ex.get("query", {}), ensure_ascii=False)
        examples_text += f"Q{i}: {ex['question']}\nA{i}: {q_str}\n\n"
    
    return f"""Ты генератор запросов к Cube.js. Преобразуй вопрос на естественном языке в валидный JSON-запрос.

## Доступные меры:
{chr(10).join(measures) if measures else 'Нет мер'}

## Доступные измерения:
{chr(10).join(dimensions) if dimensions else 'Нет измерений'}

## Формат запроса:
{{
  "measures": ["cube.measure"],
  "dimensions": ["cube.dimension"],
  "filters": [
    {{"member": "cube.field", "operator": "equals|contains|gt|lt|set|notSet", "values": ["value"]}}
  ],
  "order": {{"cube.measure": "desc"}},
  "limit": 100
}}

### Операторы: equals, contains, gt, lt, set (IS NOT NULL), notSet (IS NULL)
Для set/notSet НЕ нужен "values".

## Примеры:
{examples_text if examples_text else 'Нет примеров'}

## Правила:
1. Используй ТОЧНЫЕ имена полей из списков выше
2. Для имён людей используй оператор "contains"
3. Всегда включай хотя бы одну меру
4. ВСЕГДА включай dimensions по которым фильтруешь в вывод
5. "покажи/список" = включай детальные dimensions (key, name, etc)
6. "сколько" = используй только count и группировочное измерение
7. Верни ТОЛЬКО JSON, без пояснений

## Вопрос:
{question}

## JSON Query:"""


def parse_llm_response(content: str) -> Dict:
    """Парсинг JSON из ответа LLM"""
    content = content.strip()
    # Убрать markdown
    if content.startswith("```"):
        lines = content.split("\n")
        content = "\n".join(lines[1:])
        if content.endswith("```"):
            content = content[:-3]
    # Типографские кавычки
    for old, new in [('\u201c', '"'), ('\u201d', '"'), ('\u00ab', '"'), ('\u00bb', '"'),
                     ('\u2018', "'"), ('\u2019', "'")]:
        content = content.replace(old, new)
    # Найти JSON
    match = re.search(r'\{[\s\S]*\}', content)
    if match:
        cleaned = match.group()
        cleaned = re.sub(r',\s*}', '}', cleaned)
        cleaned = re.sub(r',\s*]', ']', cleaned)
        return json.loads(cleaned)
    return json.loads(content)


def call_cube(query: Dict) -> Dict:
    """Выполнить запрос к Cube REST API"""
    headers = {"Content-Type": "application/json"}
    if CUBE_TOKEN:
        headers["Authorization"] = f"Bearer {CUBE_TOKEN}"
    
    client = httpx.Client(timeout=30.0)
    url = f"{CUBE_URL}/load"
    resp = client.post(url, json={"query": query}, headers=headers)
    return resp.json()


def format_result(result: Dict) -> str:
    """Форматировать результат с русскими названиями"""
    if "error" in result:
        return f"❌ Ошибка: {result['error']}"
    data = result.get("data", [])
    if not data:
        return "📊 Данные не найдены"
    
    lines = [f"📊 Результаты ({len(data)} строк):"]
    for i, row in enumerate(data[:MAX_ROWS], 1):
        parts = []
        for k, v in row.items():
            name = TITLE_MAP.get(k, k.split('.')[-1])
            if isinstance(v, float):
                v = round(v, 2)
            if v is None:
                v = "—"
            parts.append(f"{name}: {v}")
        lines.append(f"  {i}. {', '.join(parts)}")
    
    if len(data) > MAX_ROWS:
        lines.append(f"  ... и ещё {len(data) - MAX_ROWS} строк")
    return "\n".join(lines)


print("✅ Ядро агента загружено")

In [ ]:
# ============================================================
# ФУНКЦИЯ ЗАПРОСА (основная)
# ============================================================

def ask(question: str, verbose: bool = False) -> str:
    """
    Задать вопрос на естественном языке.
    
    Параметры:
        question: Вопрос (на русском)
        verbose: Показать детали (запрос, промпт и т.д.)
    
    Возвращает: Форматированный ответ
    """
    start = time.time()
    
    # 1. Поиск по FAISS
    relevant = search_relevant(question)
    
    if verbose:
        print("🔎 Найдено members:")
        for m in relevant[:5]:
            print(f"   {m['score']:.1f} | {m['name']:40} | {m['title']}")
        print()
    
    # 2. Проверка уверенности
    if not relevant or relevant[0]["score"] > 18.0:
        return "🤔 Не удалось найти подходящие данные. Попробуйте переформулировать вопрос."
    
    # 3. Генерация Cube-запроса через LLM
    prompt = build_prompt(question, relevant)
    
    if verbose:
        print(f"📝 Промпт: {len(prompt)} символов")
    
    response = llm.invoke(prompt)
    
    if verbose:
        print(f"🤖 LLM ответ: {response.content[:200]}")
        print()
    
    try:
        cube_query = parse_llm_response(response.content)
    except Exception as e:
        return f"❌ Ошибка парсинга ответа LLM: {e}\n\nОтвет: {response.content[:300]}"
    
    if verbose:
        print("📋 Cube Query:")
        print(json.dumps(cube_query, ensure_ascii=False, indent=2))
        print()
    
    # 4. Валидация
    if not cube_query.get("measures"):
        return "🤔 Не удалось определить метрику. Уточните вопрос."
    
    # 5. Выполнение
    try:
        result = call_cube(cube_query)
    except Exception as e:
        return f"❌ Ошибка Cube: {e}"
    
    elapsed = int((time.time() - start) * 1000)
    answer = format_result(result)
    
    if verbose:
        print(f"⏱️ Время: {elapsed}ms")
    
    return answer


print("✅ Готово! Используйте функцию ask() для запросов.")
print()
print('Примеры:')
print('  ask("Сколько задач по проектам?")')
print('  ask("Покажи задачи по проекту AI", verbose=True)')

In [ ]:
# ============================================================
# ЗАДАВАЙТЕ ВОПРОСЫ ЗДЕСЬ
# ============================================================

print(ask("Сколько задач по проектам?"))

In [ ]:
# Ещё вопрос (дублируйте эту ячейку для новых вопросов)
print(ask("Покажи задачи по проекту AI"))

In [ ]:
# Подробный режим — показывает все шаги
print(ask("Сколько открытых задач с исполнителем Lisa", verbose=True))